<a href="https://colab.research.google.com/github/mohanrajmit/DL-Training/blob/master/expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/FER.zip

Archive:  /content/drive/MyDrive/FER.zip
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from imutils import paths
import cv2

In [ ]:
imagePaths = list(paths.list_images("/content/train"))
print(imagePaths)

In [ ]:
test_imagePaths = list(paths.list_images("/content/test"))
print(test_imagePaths)

In [ ]:
print(len(test_imagePaths))

In [ ]:
def load(imagePaths):
  # initialize the list of features and labels
  data = []
  labels = []

  # loop over the input images
  for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label assuming
    # that our path has the following format:
    # /path/to/dataset/{class}/{image}.jpg
    image = cv2.imread(imagePath,0)
    label = imagePath.split("/")[-2]
    labels.append(label)
    #label = imagePath.split(os.path.sep)[-2]
    image = cv2.resize(image, (48, 48),interpolation=cv2.INTER_AREA)

    # treat our processed image as a "feature vector"
    # by updating the data list followed by the labels
    data.append(image)
    #labels.append(label)

    # show an update every `verbose` images
    #if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
    #print("[INFO] processed {}/{}".format(i + 1,
    #len(imagePaths)))

    # return a tuple of the data and labels
  return (np.array(data), np.array(labels))

In [ ]:
print(len(imagePaths))

In [ ]:
(data, labels) = load(imagePaths)

In [ ]:
(test_data, test_labels) = load(test_imagePaths)

In [ ]:
print(test_data.shape)

In [ ]:
print(data.shape)

In [ ]:
print(labels.shape)

In [ ]:
print(labels[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le_labels = le.fit_transform(labels)

In [ ]:
test_le_labels = le.fit_transform(test_labels)

In [ ]:
le_labels

In [ ]:
le_labels.shape

In [ ]:
test_le_labels.shape

In [ ]:
y_train = to_categorical(le_labels)



In [ ]:
y_test =  to_categorical(test_le_labels)

In [ ]:
y_test[0]

In [ ]:
test_labels[0]

In [ ]:
test_le_labels[0]

In [ ]:
labels[0]

In [ ]:
le_labels[0]

In [ ]:
y_train[0]

In [ ]:

data = data.astype('float32')

data/=255

In [ ]:

test_data = test_data.astype('float32')

test_data/=255

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(48,48,1),padding='same', activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(7,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(data, y_train, batch_size=128, epochs=10,validation_data=(test_data,y_test))

In [ ]:
# save model to json
model_json = model.to_json()
with open("face_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save("face_model.h5")